In [1]:
import secml_malware
import magic
from secml.array import CArray
import shutil
import torch
import numpy as np
import os
import configparser
from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel

net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

In [2]:
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')
pm = 'predicted_malware'
malware_files = config.get(pm, 'malware_files')
output = config.get(pm, 'pr_malware')
if os.path.exists(output):
    shutil.rmtree(output)
os.makedirs(output, exist_ok=True)

for i, f in enumerate(os.listdir(malware_files)):
    path = os.path.join(malware_files, f)
    with open(path, "rb") as file_handle:
        code = file_handle.read()
        
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(),256, False)
    _, confidence = net.predict(CArray(x), True)
    if confidence[0, 1].item() > 0.5:
        destination_path = os.path.join(output, f)
        shutil.copy(path, destination_path)
        